## $NO_2$ over Madrid

In this notebook, we will use the tools introduced in ```initial_examples```to plot a map of the NO_2 mean concentration during the Covid-19 intervention over Madrid

In [120]:
import ee
import folium

from IPython.display import Image

ee.Initialize()

In [121]:
# This is a helper function to plot over folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
    ).add_to(self)

def plot_ee_data_on_map(dataset,column,begin_date,end_date,lat,lon,zoom):
    folium.Map.add_ee_layer = add_ee_layer
    vis_params = {
      'min': 0,
      'max': 0.0002,
      'opacity' : 0.5,
      'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}
    my_map = folium.Map(location=[latitude,longitude], zoom_start=zoom, height=500)
    
    # This is the core logic, where we add the computations (just the mean after filtering by dates)
    
    s5p = ee.ImageCollection(dataset).filterDate(
        begin_date, end_date)
    my_map.add_ee_layer(s5p.mean().select(column), vis_params, 'NO_2')
    
    my_map.add_child(folium.LayerControl())
    display(my_map)

In [122]:
dataset = "COPERNICUS/S5P/NRTI/L3_NO2"
column = 'NO2_column_number_density'

begin_date = '2020-03-13'
end_date = '2020-03-20'
latitude = 40.40
longitude = -3.71
zoom = 12

plot_ee_data_on_map(dataset,column,begin_date,end_date,latitude,longitude,zoom)

You should see something like this
![no2_1.png](img/no2_2.png)

Now, we will plot over the week previous to the intervention...

In [123]:
begin_date = '2020-03-05'
end_date = '2020-03-12'
latitude = 40.40
longitude = -3.71
zoom = 12

plot_ee_data_on_map(dataset,column,begin_date,end_date,latitude,longitude,zoom)

You should see something like this
![no2_1.png](img/no2_1.png)

As we can see, there has been a significative reduction in $NO_2$ concentration over Madrid after the intervention (decreased traffic). 

## Further work

To further quantify the causal impact of traffic over $NO_2$ quality, we would need the following:

* A "control" city, which hasn't received the intervention during the same period, for instance NYC, London?

* In order to account for confounding variables such as wind, proximity to the sea, ... we would also need to look for intervened cities, such as BCN...

* Try to identify more correlations: park proximity, parkings, average income...


Thus, we would have two groups of cities:

* $TREATED = \lbrace Madrid, Barcelona, ... \rbrace$

* $CONTROL = \lbrace NYC, London, ... \rbrace$

Then, we could estimate the average treatment effect $ATE$ (https://en.wikipedia.org/wiki/Average_treatment_effect) as

$$
ATE = \mathbb{E}_{x \sim TREATED}  [ Y(x) ] - \mathbb{E}_{x \sim CONTROL}  [ Y(x) ]
$$

where $Y(x)$ is the difference in average $NO_2$ between the two studied weeks for city $x$.

If the previous estimation is reasonable, then we could answer the question: how much $NO_2$ could we reduce by limiting traffic?